# Consuming TCAT API from Python

So, a student was wondering if Python could be used to ease the burden of moving stuff from [TCAT](https://github.com/digitalmethodsinitiative/dmi-tcat) to Tableau. We provide a TCAT instance and access to the `analysis` interface, where students groups have their own query bins running queries they have designed for their projects.

![](https://i.ytimg.com/vi/vRshQcRqEUI/maxresdefault.jpg)

Dangernoodle *1* vs. daggermitten *0*

## A status quo

Currently the students browse to the analysis interface, export a CSV file, and load it up in Tableau, possibly overriding some of the settings, depending on their locale (e.g. a comma `,` is a decimal separator in the Danish locale). No problem, but loads of button pressing for the humans.

## A dream

Could we re-negotiate the configuration of actors, and translate some of the labour to some software actors, by question autonomy of actors we call 'TCAT', 'Tableau' or 'Student'? Sure thing! Below is a sketch for one of many possible solutions, as a Notebook. We could translate as a cronjob, a microservice, a database trigger or something else instead, or hire an assistant, acquire an intern, or set it up as a Mechanical Turk task, or redesign our workflow to just do the data transport less frequently. Here's one idea for pedagogical and illustrative purposes, as well as for me to learn about the TCAT API and for myself to have a bit of fun.

# Overview

There are many ways to push data around between TCAT and Tableau, of course. Here, we connect to the not-very-well-known TCAT API, export all the tweets of a query bin, read it into Pandas DataFrame for SQL convenience, store it in a MySQL database, and then connect Tableau to that database.

As a bonus, we get the opportunity to poke around the data in Python, since it's brought in. That's what I personally prefer :)

## Setup for connecting to TCAT

In [1]:
import configparser
import json
import urllib.request
import pandas as pd
import sqlalchemy

Read configuration from [`tcatdb.ini`](tcatdb.ini) which describes to APIs we will use: TCAT and the database. Briefly, it looks like this:

    [tcat]
    endpoint = http://yourtcatserver/api/querybin.php
    user = adminuseraccount
    ⋮
    
    [db]
    engine = mysql
    server = databaseserverhost
    ⋮

In [2]:
config = configparser.ConfigParser()
config.read("tcatdb.ini")

['tcatdb.ini']

In [3]:
endpoint = config["tcat"]["endpoint"]
querybin = "NavCom2017_08_gang_wars"
controller = "tweets"

In [4]:
uri = "/".join([endpoint, querybin, controller])

Set up HTTP authentication. Our TCAT instance API is exposed only teaching staff, actually.

In [5]:
authinfo = urllib.request.HTTPBasicAuthHandler()
authinfo.add_password(realm=config["tcat"]["realm"],
                      uri=config["tcat"]["endpoint"],
                      user=config["tcat"]["user"],
                      passwd=config["tcat"]["user"])
opener = urllib.request.build_opener(authinfo)
opener = urllib.request.build_opener()

The above does not actually work as imagined, and I struggled with it. I believe it is because the `urllib.request.HTTPBasicAuthHandler` does not seem to [Base64 encode](https://en.wikipedia.org/wiki/Basic_access_authentication#Client_side) the `%s:%s % (user, password)` string, though I might be mistaken about the issue. Anyway, we will add the `Authorization` HTTP header ourself.

In [6]:
creds = urllib.request.base64.encodebytes(bytes("%s:%s" % (config["tcat"]["user"], config["tcat"]["passwd"]), "utf-8")).decode().strip()
opener.addheaders.append(("Authorization", "Basic " + creds))

In [7]:
[h for h, v in opener.addheaders]

['User-agent', 'Authorization']

The setup is now complete, let's read data about the query bin, and extract it's contents and store them as a Pandas DataFrame.

## Export tweets from TCAT

We will make to HTTP requests, the first activates the default `bin-info` action of the API, the second the `tweet-export` action. We might want to date-limit the extraction.

In [8]:
exportparams = urllib.parse.urlencode({
    'action': 'tweet-export',
    # 'startdate': '2017-10-05+19:19:00',
    # 'enddate': '2017-10-27+13:17:26',
    'format': 'csv'})

In [10]:
bres = opener.open(uri)
tres = opener.open(uri + "?%s" % exportparams)

Let's process the two requests.

In [61]:
bininfo = json.loads(bres.read())

In [62]:
if tres.code == 200:
    tw = pd.read_csv(tres) # A HTTPResponse is an IO object
    print(tw.shape)

(18, 36)


In [57]:
print(f"Loaded {len(tw)} of {bininfo['number-selected-tweets']} tweets.")

Loaded 18 of 18 tweets.


In [58]:
tw.sample(1)

,id,time,created_at,from_user_name,text,filter_level,possibly_sensitive,withheld_copyright,withheld_scope,truncated,...,from_user_utcoffset,from_user_timezone,from_user_lang,from_user_tweetcount,from_user_followercount,from_user_friendcount,from_user_favourites_count,from_user_listed,from_user_withheld_scope,from_user_created_at
5,919121497536557056,1507970616,2017-10-14 08:43:36,AndersEKrag,Sådan @mkrasnik 😊👏😊👏😊 @weekendavisen #dkpol #b...,low,0.0,NaN,NaN,0,...,7200.0,Copenhagen,en,31799,1113,971,14599,45,NaN,2010-02-10 06:59:38


Now that we have the TCAT query bin contents in a DataFrame, let's 

## Set up database connection

I am using `mysqlclient` as a driver for SQLAlchemy. We will have set up a database user on the server, allowed it to connect from wherever this Jupyter kernel is running and granted it sufficient permissions, and created an empty database. I did something like this, from MySQL:

    CREATE USER 'tcat'@'192.168.1.61' IDENTIFIED BY 'yeahgreatsecrethere';
    GRANT ALL PRIVILEGES ON tcat.* TO 'tcat'@'192.168.1.61';

Then we prepare a database engine

In [88]:
engine = sqlalchemy.create_engine("%s://%s:%s@%s/%s" % (
                                  config["db"]["engine"],
                                  config["db"]["user"],
                                  config["db"]["passwd"],
                                  config["db"]["server"],
                                  config["db"]["db"]))

## Thank you Pandas 🐼 for being powerful

In [90]:
tw.to_sql("tweets", engine)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/MySQLdb/cursors.py:318: Warning: (1366, "Incorrect string value: '\\xF0\\x9F\\x98\\x8A\\xF0\\x9F...' for column 'text' at row 6")
  rows += self.execute(sql + postfix)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/MySQLdb/cursors.py:318: Warning: (1366, "Incorrect string value: '\\xF0\\x9F\\x98\\x8A\\xF0\\x9F...' for column 'text' at row 7")
  rows += self.execute(sql + postfix)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/MySQLdb/cursors.py:318: Warning: (1366, "Incorrect string value: '\\xF0\\x9F\\x98\\x8A\\xF0\\x9F...' for column 'text' at row 8")
  rows += self.execute(sql + postfix)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/MySQLdb/cursors.py:318: Warning: (1366, "Incorrect string value: '\\xF0\\x9F\\x98\\x8A\\xF0\\x9F...' for column 'text' at row 9")
  rows += self.execute(sql + postfix)
/Library/Framewo

Done.

## Connect Tableau to the database

Next we want to open up Tableau and connect it to the above database.

I'm using MySQL database server.

![](Tableau connect to a MySQL server.png)

![](Tableau define MySQL database connection.png)

The server might contain more than one database, possibly something you have generated, so select the one where you stored tweets from TCAT.

![](Tableau select database from server.png)

The live connection is the default.

![](Tableau live connection.png)

You are welcome.

![](Tableau outcome.png)

# Discussion

Many other things could be done, either additionally or instead of what is presented above.

Maybe instead of a database, we want to simplify the setup just want to keep overwriting a CSV file, and have Tableau re-read it. While the data is in Python, we might want to do processing of it, or maybe create more database tables. Or we might utilize `ipywidgets` and allow the user to interactively select a subset of tweets to extract. Or we might want to engineer this out of the narrative constrain of a Jupyter Notebook and write it as a "real" program. We might want to add robustness and engineer robustness and recovery. TCAT also uses a database for it's own purposes, so of course we could just connect to it directly with Tableau and run SQL directly against it. We might want to question the whole idea of scraping Twitter and navigate ourselves towards more interesting areas of research perhaps, or ask if any of the users have given informed consent to us creeping on them.

Or we might want to do something else entirely, like enjoy the day in the park with friends.